# **Geopolitical risk uncertainty and oil future volatility: Evidence from MIDAS models**

**Etapes** :
- Création de mesures de volatité et décomposition de l'indice GPR
- Estimation et prédictions avec les modèles MIDAS à différents horizons
- Evaluation des résultats via affichage de la HMSE, HMAE et p-values des tests MCS de comparaison des modèles 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.volatility_measures import RealizedVolatilityMeasure
from src.gpr_measures import GPRDecomposer
from src.midas_models import MIDASModel
from src.model_evaluation import ModelEvaluation

## Importation des données

Data E-mini Crude Oil Futures

In [ ]:
df_oil = pd.read_csv('data/raw/E_mini_Light_Crude_Oil_Futures_4h_2018_2025.csv', delimiter=',', decimal='.')
df_oil['time'] = pd.to_datetime(df_oil['time'])
df_oil.index = df_oil['time']
df_oil.drop(columns=['time'], inplace = True)
df_oil.head()

Data indice GPR

In [ ]:
df_gpr = pd.read_excel('data/raw/data_gpr_daily_recent.xls', usecols=range(9))
df_gpr['DAY'] = pd.to_datetime(df_gpr['DAY'], format='%Y%m%d')
df_gpr.index = df_gpr['DAY']
df_gpr.drop(columns=['DAY'], inplace = True)
print('len(df_gpr) :', len(df_gpr))

## Mesures de volatilité

In [ ]:
rv_measure = RealizedVolatilityMeasure(df_oil)

# calcule de toutes les mesures de volatilité
df_volatility = rv_measure.all_volatility_measures(alpha=0.05, price_col='close')

# sauvegarde
# df_volatility.to_csv('data/result/volatility_data.csv')
# df_volatility = pd.read_csv('data/result/volatility_data.csv', index_col = 0)

df_volatility.head()

Visualisation

In [ ]:
# rv
df_volatility['rv'].plot(title='Realized Volatility 01-01-2018 à 17-05-2025')
plt.xticks(rotation=90)
plt.show()

# crv x cj
plt.figure(figsize=(8, 5))
df_volatility_2025 = df_volatility[df_volatility.index > '2025-01-01']
df_volatility_2025[['crv', 'cj']].plot(kind='area', stacked=True)
plt.xticks(rotation=90)
plt.title('Continuous component vs. Jump component - Années 2025')
plt.show()

Sur le 1er graphique, on peut observer la période de forte volatilité lors de la pandémie de COVID-19. Sur le 2nd graphique qui ne représente que l'année 2025, on peut identifier une période de volatilité plus élevée qui a suivi le 2 avril 2025 ("Liberation day"), date de début de la hausse des tarifs douaniers américains.

## Décomposition de l'indice GPR

In [ ]:
decomposer = GPRDecomposer()
df_gpr_augmente = decomposer.fit_transform(df_gpr['GPRD'])

- gpr (Geopolitical Risk index): Indice général mesurant les risques géopolitiques mondiaux
- GPRD_ACT (Geopolitical Risk - Actions): Mesure des risques géopolitiques basée sur les actions concrètes des acteurs politiques (press coverage of actual adverse geopolitical events, such as terrorist acts or the beginning of a war)
- GPRD_THREAT (Geopolitical Risk - Threats): Mesure des risques géopolitiques basée sur les menaces et déclarations d'intention.
- gpr_expected (Expected Geopolitical Risk): Mesure de risques géopolitiques qui ne sont pas imprévus (découle du lag d'un modèle AR(1))
- gpr_shocked (Shocked Geopolitical Risk): Mesure des chocs ou surprises de risques géopolitiques imprévus (découle du résidu d'un modèle AR(1))

In [ ]:
# on veut garder GPRD_ACT et GPRD_THREAT pour la suite
df_gpr_augmente = pd.merge(df_gpr_augmente, df_gpr[['GPRD_ACT', 'GPRD_THREAT']], how='left', left_index=True, right_index=True)
df_gpr_augmente.head()

## **Modèles MIDAS** : Estimations et prédictions

Construction du df joint

In [ ]:
# df des volatilités du E-mini Crude Oil Futures
df_volatility.index = pd.to_datetime(df_volatility.index).normalize()
df_volatility.index = df_volatility.index.tz_localize(None)
print('len(df_volatility) :', len(df_volatility))

# df complet : mesures de volatilités + indices GPR
df = df_volatility.join(df_gpr_augmente, how='left')
df = df[:-1] # suppression  de la dernière ligne
print('len(df) :', len(df))
df.head()

train test split

In [ ]:
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]
print('shape train_df :', train_df.shape)
print('shape test_df :', test_df.shape)

Choix de l'indice GPR

In [ ]:
gpr_str = 'gpr' # gpr ou gpr_expected ou gpr_shocked ou GPRD_THREAT ou GPRD_ACT

Choix des horizons à prédire

In [ ]:
horizons = [1, 5, 22]

Choix de maxiter et kmax

In [ ]:
kmax = 66
maxiter = 1000

Lancement du training et de la prédiction sur l'echantillon test

In [ ]:
results = {}

# estimation par horizon
for h in horizons:
    print("\n", "-"*50, "prédiction à", h, "jours", "-"*50)
    
    # preparation des variables cibles pour chaque horizon (comme à la fin de la 2eme page de l'article)
    # en prenant la moyenne des h prochaines valeurs de RV pour l'horizon h
    # pour le train
    target = {}
    for i in range(len(train_df) - h):
        if i+h < len(train_df):
            # moyenne des h prochaines valeurs de RV pour l'horizon h
            target[i] = train_df['rv'].iloc[i+1:i+h+1].mean()
    
    y_train = np.array(list(target.values()))

    # pour le test
    target_test = {}
    for i in range(len(test_df) - h):
        if i+h < len(test_df):
            target_test[i] = test_df['rv'].iloc[i+1:i+h+1].mean()
    
    y_test = np.array(list(target_test.values()))
    
    # Ajustement du modèle MIDAS-RV (Modèle 0)
    print("Ajustement du modèle MIDAS-RV (Modèle 0)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    params_rv = midas.fit_midas_rv(y_train,
                                    train_df['rv'].values,
                                    horizon=h)
    print(f"Paramètres estimés: {params_rv}")
    
    pred_rv = midas.predict(test_df, model_type='MIDAS-RV')
    

    # Ajustement du modèle MIDAS-RS (Modèle 1)
    print("Ajustement du modèle MIDAS-RS (Modèle 1)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    
    params_rs = midas.fit_midas_rs(y_train,
                                    train_df['rv_pos'].values,
                                    train_df['rv_neg'].values,
                                    horizon=h)
    print(f"Paramètres estimés: {params_rs}")

    pred_rs = midas.predict(test_df, model_type='MIDAS-RS')
    
    # Ajustement du modèle MIDAS-CJ (Modèle 2)
    print("Ajustement du modèle MIDAS-CJ (Modèle 2)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    params_cj = midas.fit_midas_cj(y_train,
                                    train_df['crv'].values,
                                    train_df['cj'].values,
                                    horizon=h)
    print(f"Paramètres estimés: {params_cj}")
    
    pred_cj = midas.predict(test_df, model_type='MIDAS-CJ')

    # Ajustement du modèle MIDAS-RV-GPR (Modèle 3)
    print("Ajustement du modèle MIDAS-RV-GPR (Modèle 3)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    params_rv_gpr = midas.fit_midas_rv_gpr(y_train,
                                            train_df['rv'].values,
                                            train_df[gpr_str].values,
                                            horizon=h)
    print(f"Paramètres estimés: {params_rv_gpr}")
    
    pred_rv_gpr = midas.predict(test_df, model_type='MIDAS-RV-GPR', gpr_str=gpr_str)

    # Ajustement du modèle MIDAS-RS-GPR (Modèle 4)
    print("Ajustement du modèle MIDAS-RS-GPR (Modèle 4)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    params_rs_gpr = midas.fit_midas_rs_gpr(y_train, 
                                            train_df['rv_pos'].values,
                                            train_df['rv_neg'].values,
                                            train_df[gpr_str].values,
                                            horizon=h)
    print(f"Paramètres estimés: {params_rs_gpr}")
    
    pred_rs_gpr = midas.predict(test_df, model_type='MIDAS-RS-GPR', gpr_str=gpr_str)

    # Ajustement du modèle MIDAS-CJ-GPR (Modèle 5)
    print("Ajustement du modèle MIDAS-CJ-GPR (Modèle 5)")
    midas = MIDASModel(kmax=kmax, options_dict = {'maxiter': maxiter})
    params_cj_gpr = midas.fit_midas_cj_gpr(y_train,
                                            train_df['crv'].values,
                                            train_df['cj'].values,
                                            train_df[gpr_str].values,
                                            horizon=h)
    print(f"Paramètres estimés: {params_cj_gpr}")
    
    pred_cj_gpr = midas.predict(test_df, model_type='MIDAS-CJ-GPR', gpr_str=gpr_str)
    
    # predictions avec les différents modèles
    predictions = {
        'MIDAS-RV': pred_rv,
        'MIDAS-RS': pred_rs,
        'MIDAS-CJ': pred_cj,
        'MIDAS-RV-GPR': pred_rv_gpr,
        'MIDAS-RS-GPR': pred_rs_gpr,
        'MIDAS-CJ-GPR': pred_cj_gpr
    }

    # Evaluation des modèles sur l'ensemble de test

    # métriques d'erreur de l'article: HMSE et HMAE
    hmse = {}
    hmae = {}
    
    for name, pred in predictions.items():

        min_len = min(len(pred), len(y_test))
        pred_aligned = pred[:min_len - 2]
        y_test_aligned = y_test[:min_len - 2]
        
        ratio = np.divide(pred_aligned, y_test_aligned, out=np.zeros_like(pred_aligned), where=y_test_aligned!=0)
        
        # HMSE (équation 14 de l'article)
        hmse[name] = np.mean((1 - ratio) ** 2)
        # HMAE (équation 15 de l'article)
        hmae[name] = np.mean(np.abs(1 - ratio))
        
        print(f"{name}: HMSE = {hmse[name]:.4f}, HMAE = {hmae[name]:.4f}")
    
    # resultats
    results[h] = {
        'hmse': hmse,
        'hmae': hmae,
        'predictions': predictions,
        'actual': y_test
    }

## **Modèles MIDAS** : Evaluation des résultats

In [ ]:
evaluation = ModelEvaluation()

### Out-of-sample evaluations : métriques HMSE et HMAE

In [ ]:
evaluation.create_loss_metrics_table(results, horizons)

### Out-of-sample evaluations : p-values des tests MCS

In [ ]:
mcs_results = evaluation.run_all_mcs_tests(results, horizons = horizons, alpha = 0.25)
evaluation.create_mcs_metrics_table(mcs_results, horizons = horizons)